In [5]:
##!python
#!/usr/bin/env python

import os
import shutil
from scipy.io import loadmat
import numpy as np
import scipy.io
from operator import itemgetter
from keras.preprocessing import image
import matplotlib.pyplot as plt
import cv2 
from tqdm import tqdm
import tensorflow as tf
import tensorflow.keras
from keras.optimizers import RMSprop
from tensorflow.keras.layers import (
                                    Dense,
                                    Conv2D, 
                                    BatchNormalization, 
                                    ReLU, 
                                    Add,
                                    Input,
                                    MaxPooling2D,
                                    UpSampling2D,
                                    )
from keras.models import Model, load_model
from keras.losses import mean_squared_error
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau
from math import exp
from sklearn.model_selection import train_test_split
import datetime
%matplotlib inline

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))

# Limpieza de Datos/Preprocesamiento del dataset

## funciones:

In [2]:
NUMBER_MIN_OF_IMGS_FOR_TRAINING = 0
NUMBER_MAX_OF_IMGS_FOR_TRAINING = 11400
########################################################################################################################################################
def generate_dataset_obj(obj):
    if type(obj) == np.ndarray:
        dim = obj.shape[0]
        if dim == 1:
            ret = generate_dataset_obj(obj[0])             
        else:
            ret = []
            for i in range(dim):
                ret.append(generate_dataset_obj(obj[i]))                

    elif type(obj) == scipy.io.matlab.mio5_params.mat_struct:
        ret = {}
        for field_name in obj._fieldnames:            
            field = generate_dataset_obj(obj.__dict__[field_name])
            if field_name in must_be_list_fields:
                field = [field]
                ret[field_name] = field

    else:
        ret = obj

    return ret
########################################################################################################################################################
def generate_dataset_obj(obj):
    if type(obj) == np.ndarray:
        dim = obj.shape[0]
        if dim == 1:
            ret = generate_dataset_obj(obj[0])             
        else:
            ret = []
            for i in range(dim):
                ret.append(generate_dataset_obj(obj[i]))                

    elif type(obj) == scipy.io.matlab.mio5_params.mat_struct:
        ret = {}
        for field_name in obj._fieldnames:            
            field = generate_dataset_obj(obj.__dict__[field_name])
            if field_name in must_be_list_fields:
                field = [field]
                ret[field_name] = field

    else:
        ret = obj

    return ret

########################################################################################################################################################
def print_dataset_obj(obj, depth = 0, maxIterInArray = 20):
    prefix = "  "*depth
    if type(obj) == dict:
        for key in obj.keys():
            print("{}{}".format(prefix, key))
            print_dataset_obj(obj[key], depth + 1)
    elif type(obj) == list:
        for i, value in enumerate(obj):
            if i >= maxIterInArray:
                break
            print("{}{}".format(prefix, i))
            print_dataset_obj(value, depth + 1)
    else:
        print("{}{}".format(prefix, obj))
########################################################################################################################################################
def return_image_joints(name,data):
    for item in data: # guardar coordenadas de los joints
        if item[0] == name:
            #print(item[1]) 
            return item[1]
########################################################################################################################################################
rightconnections = [
                    (0,1),(1,2),(3,4),(4,5),(2,6),
                    (3,6),(6,7),(7,8),(8,9),(10,11),
                    (11,12),(12,7),(13,7),(13,14),(14,15)
                   ]
size_img_x = 256
size_img_y = 256
def draw_img_joints(file_name, data, resize = False ):    
    # Load image
    #img = cv2.imread(Path_To_Single_Person_Images + "/" + file_name,1)  
    img = image.load_img(Path_To_Single_Person_Images + "/" + file_name)
    img = image.img_to_array(img) 
    img = img/255
    if resize:
        img = np.float32(tf.image.resize(img,(size_img_x, size_img_y)))  
    pts = return_image_joints(file_name, data)        
    #plt.imshow(img)  
    X = [x[0] for x in pts]
    Y = [y[1] for y in pts]
    X = [int(x) for x in X]
    Y = [int(y) for y in Y]
    
    for i in range(16):
        for j in range(16):
            if (i,j) in rightconnections:
                if X[i]>0 and X[j]>0 and Y[i]>0 and Y[j]>0:
                    img = cv2.line(img,(X[i],Y[i]),(X[j],Y[j]),(1,0,0),5)
                    plt.scatter(X[i], Y[i], marker="o", color="red", s=20)
                    plt.scatter(X[j], Y[j], marker="o", color="red", s=20)
                    
    plt.imshow(img)
########################################################################################################################################################
def crop_resize(imagen, joints, scale, margin = 0.2):
    # CROP PART
    xmin = 9999
    ymin = 9999
    xmax = -1
    ymax = -1
    body_height_margin = scale * 200 * margin
    img_height = imagen.shape[0]
    img_width = imagen.shape[1]
    for item in joints:
        if item[0] >= 0 and item[0] < xmin : xmin = item[0]
        if item[1] >= 0 and item[1] < ymin : ymin = item[1] 
        if item[0] >= 0 and item[0] > xmax : xmax = item[0]
        if item[1] >= 0 and item[1] > ymax : ymax = item[1]    
    xmin = int(xmin - body_height_margin)
    ymin = int(ymin - body_height_margin)
    xmax = int(xmax + body_height_margin)
    ymax = int(ymax + body_height_margin)
    if xmin < 0: xmin = 0
    if ymin < 0: ymin = 0
    if xmax > img_width: xmax = img_width
    if ymax > img_height: ymax = img_height
    imagen = imagen[ymin:ymax, xmin:xmax, :]
    #print("xmin:",xmin,", ymin: ",ymin,", xmax: ",xmax,", ymax: ",ymax)
    #RESIZE PART
    img_new_height = imagen.shape[0]
    img_new_width = imagen.shape[1]
    scala_x = img_new_width / size_img_x
    scala_y = img_new_height / size_img_y
    for i in range(16): # escala los puntos clave
        joints[i] = np.array([(joints[i][0] - xmin) / scala_x, (joints[i][1] - ymin) / scala_y]) 
        #print("x:",joints[i][0],", y: ",joints[i][1])
    imagen = tf.image.resize(imagen,(size_img_x, size_img_y))       
    imagen = imagen/255
    
    return imagen
########################################################################################################################################################    
def load_image(train_data, a, b):
    train = np.asarray(train_data[a:b])
    train_image = np.zeros((b-a,size_img_x,size_img_y,3))
    #print(train[2])
    for i in tqdm(range(a,b)):
        name_img = train[i][0]
        img = image.load_img(Path_To_Single_Person_Images + '/' + name_img)
        img = image.img_to_array(img) 
        #crop and resize
        train_image[i] = crop_resize(img, train[i][1], train[i][2])
    return train_image, train
########################################################################################################################################################
def MakeHeatmap(x, y, width, height, show = False):
    # Probability as a function of distance from the center derived
    # from a gaussian distribution with mean = 0 and stdv = 1
    scaledGaussian = lambda x : exp(-(1/2)*(x**2))

    imgSize = (height, width)
    center_x = int(round(x)) #redondeamos
    center_y = int(round(y))

    isotropicGrayscaleImage = np.zeros((imgSize[0],imgSize[1]),np.uint8)
    
    if center_x > 0 and center_y > 0 :
        for i in np.unique(np.clip(np.array(range(center_y-3,center_y+4,1)),0,height-1)): #solo queremos calcular un parche de 7x7, ademas debemos evitar valores fuera de la matriz, finalmente quitamos valores repetidos
            for j in np.unique(np.clip(np.array(range(center_x-3,center_x+4,1)),0,width-1)):

                # find euclidian distance from center of image (x,y) 
                # and scale it to range of 0 to 2.5 as scaled Gaussian
                # returns highest probability for x=0 and approximately
                # zero probability for x > 2.5

                distanceFromCenter = np.linalg.norm(np.array([i-center_y,j-center_x]))
                #distanceFromCenter = 18*distanceFromCenter/(imgSize/2)
                scaledGaussianProb = scaledGaussian(distanceFromCenter)                
                isotropicGrayscaleImage[i,j] = np.clip(scaledGaussianProb*255,0,255) 
                
        return isotropicGrayscaleImage
    else: 
        return isotropicGrayscaleImage
########################################################################################################################################################    
def Joints_heatmaps(lista_de_joints, heatmap_size_x, heatmap_size_y, num_heatmaps = 16, show = False):
    heatmaps = np.zeros((16,64,64))
    for i in range(num_heatmaps):
        x, y = lista_de_joints[i] 
        x = x / 4 # entre 4 por que el array es de 256x256
        y = y / 4 # entre 4 por que el array es de 256x256
        heatmaps[i] = MakeHeatmap(x, y, heatmap_size_x, heatmap_size_y)
    if show:
        plotImages(heatmaps, num_heatmaps)
    return heatmaps
########################################################################################################################################################        
def plotImages(images_arr, num_images):
    fig, axes = plt.subplots(1, num_images, figsize=(60,60))
    axes = axes.flatten()
    for img, ax in zip(images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()
########################################################################################################################################################    

## Cargar datos, Generación de heatmaps

In [3]:
#divimos la data para ingresarla al modelo,
if os.path.isfile('X_train.npy') and os.path.isfile('X_test.npy') and os.path.isfile('y_train.npy') and os.path.isfile('y_test.npy'):    
    #Primero comprobamos si existen ya datos para usar en el modelo, si hay los mapeamos, no se cargan a la ram, se leen desde disco 
    X_train = np.load('X_train.npy', mmap_mode='r')
    X_test = np.load('X_test.npy', mmap_mode='r')
    y_train = np.load('y_train.npy', mmap_mode='r')
    y_test = np.load('y_test.npy', mmap_mode='r')  
else:      
    # si no existen entonces iniciamos el preprocesado:   
    ########################################################################################################################################################
    if os.path.isfile('lista_de_imagenes.npy') and os.path.isfile('lista_de_heatmaps.npy'):    
        #Ahora cargamos las imagenes y sus heatmaps
        lista_de_heatmaps = np.load('lista_de_heatmaps.npy', mmap_mode='r')  
        lista_de_imagenes = np.load('lista_de_imagenes.npy', mmap_mode='r')    
    else: 
        ########################################################################################################################################################
        #darle formato de diccionario
        matph = './mpii.mat'
        decoded1 = loadmat(matph, struct_as_record=False)["RELEASE"]
        must_be_list_fields = ["annolist","image","name", "annorect", "scale", "x", "y", "annopoints", "point", "id"]
        # Convert to dict
        dataset_obj = generate_dataset_obj(decoded1)
        # Print it out
        #print_dataset_obj(dataset_obj)
        len(dataset_obj['annolist'][0])
        #solo queremos la información en 'annolist'
        dataset = dataset_obj['annolist'][0]
        ########################################################################################################################################################
        #guardamos solo informacion de las imagenes que tienen solo una persona
        train_data = []
        for i in range(len(dataset)):
            if "annopoints" in dataset[i]['annorect'][0]:     
                name = dataset[i]['image'][0]['name'][0]
                scale = dataset[i]['annorect'][0]['scale'][0]
                tupla = np.full((16, 3), -1) #creo un prototipo de array de joints lleno de -1 si la articulación es visible se reemplaza el -1
                for j in range(len(dataset[i]['annorect'][0]['annopoints'][0]['point'][0])):     #ordena los puntos de articulaciones del id = 0 al id = 15       
                    try:
                        x = dataset[i]['annorect'][0]['annopoints'][0]['point'][0][j]['x'][0]
                    except:
                        x = -1
                    try:
                        y = dataset[i]['annorect'][0]['annopoints'][0]['point'][0][j]['y'][0]
                    except:
                        y = -1
                    try:
                        joint_id = dataset[i]['annorect'][0]['annopoints'][0]['point'][0][j]['id'][0]
                    except:
                        joint_id = -1          
                    tupla[joint_id] = np.array([x,y,joint_id]) # esto lo ordena
                #tupla = sorted(tupla, key = itemgetter(2)) 
                #tupla = tupla[np.argsort(tupla[:, 2])] 
                tupla = np.delete(tupla, 2, 1)  #quita id de las tuplas,                    
                #pasa de tupla a array
                tupla = np.asarray(tupla)        
                train_data.append((name,tupla,scale))
        #Creamos un array para guardar los nombres        
        names = []
        for item in train_data:#guardar nombres de las imagenes que voy a usar en "name"
            names.append(item[0])
        ########################################################################################################################################################
        #Crear Carpeta para guardar imagenes del dataset
        Path_To_Raw_Images = 'DataSet/mpii_human_pose_v1_images'
        Path_To_Single_Person_Images = 'DataSet/mpii_human_pose_v1_images/SinglePersonImagesWithData'
        os.chdir(Path_To_Raw_Images)

        if os.path.isdir('SinglePersonImagesWithData') is False:
            os.makedirs('SinglePersonImagesWithData')
            for images in names:
                shutil.move(images, 'SinglePersonImagesWithData')

        os.chdir('../../')

        #Demostración dibujar joints en imagenes con data
        #draw_img_joints('060111501.jpg',train_data)
        ########################################################################################################################################################
        #Ahora cargamos las imagenes
        lista_de_imagenes, lista_de_joints = load_image(train_data,NUMBER_MIN_OF_IMGS_FOR_TRAINING,NUMBER_MAX_OF_IMGS_FOR_TRAINING)
        np.save('lista_de_imagenes', lista_de_imagenes)
        ########################################################################################################################################################
        #Ahora creamos los heatmaps para lista_de_imagenes:
        heatmap_size_x = 64
        heatmap_size_y = 64 
        #dibujo_test_heatmaps = Joints_heatmaps(lista_de_joints[0][1], heatmap_size_x, heatmap_size_y, show = False)
        #creamos los heatmaps de nuestra data
        lista_de_heatmaps = np.zeros((NUMBER_MAX_OF_IMGS_FOR_TRAINING - NUMBER_MIN_OF_IMGS_FOR_TRAINING,heatmap_size_x,heatmap_size_y,16))
        for i in tqdm(range(lista_de_joints.shape[0])):
            joints = return_image_joints(lista_de_joints[i][0], lista_de_joints)
            lista_de_heatmaps[i] = np.moveaxis(Joints_heatmaps(joints, heatmap_size_x, heatmap_size_y), 0, -1) # change shape from 16x64x64 to 64x64x16
        #guardamos el array    
        np.save('lista_de_heatmaps', lista_de_heatmaps)
        #liberamos memoria cargando los archivos desde disco
        lista_de_heatmaps = np.load('lista_de_heatmaps.npy', mmap_mode='r')  
        lista_de_imagenes = np.load('lista_de_imagenes.npy', mmap_mode='r') 
        #plotImages(lista_de_heatmaps[67], 16)
    ######################################################################################################################################################## 
    X_train, X_test, y_train, y_test = train_test_split(lista_de_imagenes, lista_de_heatmaps, random_state=7, test_size=0.2)
    #guardamos en disco y liberamos ram
    np.save('X_train', X_train)
    X_train = np.load('X_train.npy', mmap_mode='r')
    np.save('X_test', X_test)
    X_test = np.load('X_test.npy', mmap_mode='r')
    np.save('y_train', y_train)
    y_train = np.load('y_train.npy', mmap_mode='r')
    np.save('y_test', y_test)
    y_test = np.load('y_test.npy', mmap_mode='r')   

In [4]:
"""###test part
imgidtest = 9
print("image name: ",lista_de_joints[imgidtest][0])
print("joints locations: \n",lista_de_joints[imgidtest][1])
print("scale: ",lista_de_joints[imgidtest][2])
plt.imshow(lista_de_imagenes[imgidtest])
plotImages(np.moveaxis(lista_de_heatmaps[imgidtest], -1, 0),16)
"""

'###test part\nimgidtest = 9\nprint("image name: ",lista_de_joints[imgidtest][0])\nprint("joints locations: \n",lista_de_joints[imgidtest][1])\nprint("scale: ",lista_de_joints[imgidtest][2])\nplt.imshow(lista_de_imagenes[imgidtest])\nplotImages(np.moveaxis(lista_de_heatmaps[imgidtest], -1, 0),16)\n'

# RED NEURONAL

In [5]:
def conv2D(inputs, 
           filters, 
           kernel_size = 1,
           strides = 1,
           padding = 'same',
           kernel_initializer = 'he_normal',
           activation = True,
           batch_normalization = True,
           name = "conv"):
    
    x = Conv2D(filters, kernel_size, strides=strides, padding=padding,
            use_bias=False, kernel_initializer=kernel_initializer)(inputs)
    if batch_normalization:
        x = BatchNormalization()(x)
    if activation:
        x = ReLU()(x)
    return x
    

In [6]:
#Arquitectura ResNetV2 de cuello de botella
def ResNetV2(inputs, filters, strides = 1, lift_channels = False, name = 'bloque'):
    
    res = inputs
    #incrementa el número de canales si es necesario
    if lift_channels:
        res = conv2D(
            inputs,
            filters,
            activation = False,
            batch_normalization = False)
    
    x = BatchNormalization()(inputs)
    x = ReLU()(x)
    
    #conv de tamaño 1x1
    x = conv2D(
        x,
        filters = filters/2)
    
    #conv de tamaño 3x3
    x = conv2D(
        x,
        filters = filters/2,
        kernel_size = 3)
    
    #conv de tamaño 1x1
    x = conv2D(
        x,
        filters = filters,
        activation = False,
        batch_normalization = False)
    
    x = Add()([res,x])
    
    return x

In [7]:
def HourglassUnit(inputs, depth, filters, resnet_per_block, name = 'hourglass_'):    
    
    #Capas "superiores"
    up_1 = ResNetV2(inputs,filters)
    
    for i in range(resnet_per_block):
        up_1 = ResNetV2(up_1, filters)
    
    #Capas "inferiores"
    #Reducir resolución
    low_1 = MaxPooling2D(pool_size = 2, strides = 2)(inputs)
    
    for i in range(resnet_per_block):
        low_1 = ResNetV2(low_1, filters)
    
    low_2 = low_1
    if depth > 1 : 
        low_2 = HourglassUnit(low_1, depth-1, filters, resnet_per_block)
    else:
        low_2 = ResNetV2(low_2, filters)
    
    low_3 = low_2
    
    for i in range(resnet_per_block):
        low_3 = ResNetV2(low_3, filters)
    
    #Aumentar resolución
    up_2 = UpSampling2D()(low_3)
    
    return Add()([up_1,up_2])
    

In [8]:
def HourglassNetwork(input_shape = (256,256,3), stacks = 8, resnet_per_block = 3, heatmaps = 16):
    
    inputs = Input(shape = input_shape)
    
    #la data llega en formato 256x256x3, la pasamos a 64x64x256
    #preprocessing
    #amplia canales a 64
    x = conv2D(
        inputs,
        filters = 64,
        kernel_size = 7,
        strides = 2)
    #amplia canales de 64 a 128 
    x = ResNetV2(x, filters = 128, lift_channels = True)
    x = MaxPooling2D(pool_size = 2, strides = 2)(x)
    x = ResNetV2(x, filters = 128)
    #amplia canales de 64 a 128
    x = ResNetV2(x, filters = 256, lift_channels = True)
    skip = x
    y_heatmaps = []
    
    for i in range(stacks):
        x = HourglassUnit(skip, depth = 4, filters = 256, resnet_per_block = resnet_per_block)
        
        x = ResNetV2(x, filters = 256)
        
        #prediccion de 256 canales 
        x = conv2D(x, filters = 256)
        
        #prediccion temporal de heatmaps
        y = conv2D(x, filters = heatmaps)
        #agregamos el resultado temportal al array de resultados para la supervision intermedia
        y_heatmaps.append(y)
        
        #ahora regresamos el tensor y al orden de 256 canales si es que no es el ultimo output
        if i < stacks - 1:
            y_recovery1 = conv2D(x, filters = 256, activation = False, batch_normalization = False)
            y_recovery2 = conv2D(y, filters = 256, activation = False, batch_normalization = False)
            skip = Add()([skip, y_recovery1, y_recovery2])
    #print(y_heatmaps)
    return Model(inputs = inputs, outputs = y_heatmaps, name = 'HourglassNetwork')

In [9]:
############################################################################
# tasa de decrecimiento del learning rate por numero de epoch
def lr_schedule(epoch): #tome esto de resnet.ipynb
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-4
    if epoch > 180:
        lr *= 1e-1
    elif epoch > 150:
        lr *= 1e-1
    elif epoch > 100:
        lr *= 1e-2
    elif epoch > 50:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr 
############################################################################
# custom loss mean_squared_error function, sirve para magnificar la presencia de todos los pixeles > que 0 en los mapas de calor
def custom_loss_mse_function(y_true, y_pred):
    weights = tf.cast(y_true > 0, dtype=tf.float32) * 81 + 1
    return tf.reduce_mean(tf.math.square(y_true - y_pred) * weights)
############################################################################

## Inicio el modelo

In [10]:
if os.path.isfile('single_person_modelvFULLDATA.h5'):
    # verificamos si hay algun punto de guardado del modelo
    model = load_model('single_person_modelvFULLDATA.h5', custom_objects={'custom_loss_mse_function': custom_loss_mse_function})
else:    
    # si no lo hay creamos el modelo desde cero
    model = HourglassNetwork(stacks = 4, resnet_per_block = 1)
    rms = RMSprop(lr=lr_schedule(0))
    model.compile(optimizer=rms, loss=custom_loss_mse_function, metrics=["accuracy"])

In [11]:
# creamos puntos de guardado del modelo para que guarde solo la mejor version durante el entrenamiento:
checkpoint = ModelCheckpoint('single_person_modelvFULLDATA.h5', monitor='re_lu_233_loss', verbose=1, save_best_only=True, mode='min')
# modificaciones al learning rate per epoch:
lr_scheduler = LearningRateScheduler(lr_schedule)
# modificaciones al learning rate por estancamiento:
lr_reducer = ReduceLROnPlateau(monitor = 'val_re_lu_233_loss',
                               factor = 0.1,
                               patience = 10,
                               min_lr = 0,
                              verbose = 1)

In [7]:
# Clear any logs from previous runs
#os.remove("./logs/")
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%d/%m/%Y - %H:%M:%S")
log_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

PermissionError: [WinError 5] Acceso denegado: './logs/'

In [13]:
# agregamos todas las funciones al callback_list
callbacks_list = [checkpoint, lr_scheduler, lr_reducer, log_callback]

In [14]:
history = model.fit(X_train, y_train, epochs=200, validation_data=(X_test, y_test), batch_size=4, callbacks = callbacks_list)

Learning rate:  0.0001
Epoch 1/10
1/4 [======>.......................] - ETA: 0s - loss: 13333.8320 - re_lu_65_loss: 3415.3728 - re_lu_121_loss: 3346.9939 - re_lu_177_loss: 3296.0701 - re_lu_233_loss: 3275.3960 - re_lu_65_accuracy: 0.1098 - re_lu_121_accuracy: 0.1277 - re_lu_177_accuracy: 0.2007 - re_lu_233_accuracy: 0.1428WARNING:tensorflow:From c:\users\espin\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
4/4 [==============================] - ETA: 0s - loss: 13122.1162 - re_lu_65_loss: 3358.2681 - re_lu_121_loss: 3301.2883 - re_lu_177_loss: 3250.9561 - re_lu_233_loss: 3211.6035 - re_lu_65_accuracy: 0.1013 - re_lu_121_accuracy: 0.1093 - re_lu_177_accuracy: 0.1690 - re_lu_233_accuracy: 0.1483
Epoch 00001: loss improved from inf to 13122.11621, saving model to sing